# Very less accuracy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Deep Q Network

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import random

class DQNAuthorizationModel:
    def __init__(self, learning_rate=0.001, discount_factor=0.95, epsilon=1.0,
                 epsilon_min=0.01, epsilon_decay=0.995, behavior_threshold=0.7):
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.behavior_threshold = behavior_threshold
        self.model = None

    def _build_model(self):
        """Build the Deep Q-Network for pre-encoded data"""
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, input_dim=3, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(2, activation='linear')  # 2 actions: deny (0) or allow (1)
        ])

        model.compile(loss='mse',
                     optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def train(self, data, test_size=0.2, epochs=10):
        """Train the DQN model with pre-encoded data"""
        print("\n=== Training Configuration ===")
        print(f"Learning Rate: {self.learning_rate}")
        print(f"Initial Epsilon: {self.epsilon}")
        print(f"Behavior Threshold: {self.behavior_threshold}")
        print(f"Total epochs: {epochs}\n")

        # Split data
        train_idx = int(len(data) * (1 - test_size))
        train_data = data[:train_idx]
        test_data = data[train_idx:]
        print(f"Data split: {len(train_data)} training samples, {len(test_data)} test samples")

        # Initialize model if not already done
        if self.model is None:
            print("\nInitializing neural network...")
            self.model = self._build_model()
            print("Model architecture:")
            self.model.summary()
            print("\n=== Starting Training ===")

        training_accuracy = []
        best_accuracy = 0

        for epoch in range(epochs):
            print(f"\nEpoch {epoch + 1}/{epochs}")
            print("-" * 20)

            correct_predictions = 0
            total_predictions = len(train_data)
            epoch_loss = 0

            for i in range(len(train_data)):
                # Get current state
                state = np.array([
                    train_data['Role_Encoded'].iloc[i],
                    train_data['File_Path_Encoded'].iloc[i],
                    train_data['Behavior_Probability'].iloc[i]
                ])

                # Epsilon-greedy action selection
                if random.random() < self.epsilon:
                    action = random.randint(0, 1)
                    action_type = "Random"
                else:
                    q_values = self.model.predict(state.reshape(1, -1), verbose=0)
                    action = np.argmax(q_values[0])
                    action_type = "Predicted"

                # Get reward
                correct_action = train_data['Access_Granted'].iloc[i]
                reward = 1.0 if action == correct_action else -1.0

                # Consider behavior probability
                if train_data['Behavior_Probability'].iloc[i] < self.behavior_threshold:
                    reward *= 0.5

                # Calculate target Q-value
                current_q = self.model.predict(state.reshape(1, -1), verbose=0)
                target_q = current_q.copy()
                target_q[0][action] = reward

                # Train on single example
                loss = self.model.train_on_batch(state.reshape(1, -1), target_q)
                epoch_loss += loss

                # Track accuracy
                predicted_access = action and (train_data['Behavior_Probability'].iloc[i] >= self.behavior_threshold)
                if predicted_access == correct_action:
                    correct_predictions += 1

                # Print progress every 1000 samples
                if (i + 1) % 1000 == 0:
                    print(f"Progress: {i+1}/{total_predictions} samples processed")
                    print(f"Current Action: {action_type}, Loss: {loss:.4f}")
                    print(f"Current Epsilon: {self.epsilon:.4f}")

                # Decay epsilon
                if self.epsilon > self.epsilon_min:
                    self.epsilon *= self.epsilon_decay

            # Calculate metrics for this epoch
            accuracy = correct_predictions / total_predictions
            avg_loss = epoch_loss / total_predictions
            training_accuracy.append(accuracy)

            print(f"\nEpoch {epoch + 1} Summary:")
            print(f"Average Loss: {avg_loss:.4f}")
            print(f"Training Accuracy: {accuracy:.4f}")
            print(f"Final Epsilon: {self.epsilon:.4f}")

            # Track best performance
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                print("New best accuracy achieved!")

        # Test the model
        print("\n=== Testing Phase ===")
        test_accuracy = self.test(test_data)

        print("\n=== Training Complete ===")
        print(f"Best Training Accuracy: {best_accuracy:.4f}")
        print(f"Final Test Accuracy: {test_accuracy:.4f}")

        return training_accuracy[-1], test_accuracy

    def test(self, test_data):
        """Test the model with pre-encoded data"""
        print("Starting model testing...")
        correct_predictions = 0
        total_predictions = len(test_data)

        for i in range(len(test_data)):
            state = np.array([
                test_data['Role_Encoded'].iloc[i],
                test_data['File_Path_Encoded'].iloc[i],
                test_data['Behavior_Probability'].iloc[i]
            ])

            q_values = self.model.predict(state.reshape(1, -1), verbose=0)
            action = np.argmax(q_values[0])

            predicted_access = action and (test_data['Behavior_Probability'].iloc[i] >= self.behavior_threshold)
            if predicted_access == test_data['Access_Granted'].iloc[i]:
                correct_predictions += 1

            if (i + 1) % 1000 == 0:
                print(f"Testing progress: {i+1}/{total_predictions}")

        accuracy = correct_predictions / total_predictions
        return accuracy

    def predict(self, role_encoded, file_path_encoded, behavior_prob):
        """Make a prediction for a single pre-encoded case"""
        state = np.array([role_encoded, file_path_encoded, behavior_prob])
        q_values = self.model.predict(state.reshape(1, -1), verbose=0)
        action = np.argmax(q_values[0])

        return action and (behavior_prob >= self.behavior_threshold)

    def save_model(self, filename='dqn_auth_model.h5'):
        """Save the model"""
        print(f"\nSaving model to {filename}...")
        model_data = {
            'model_weights': self.model.get_weights(),
            'params': {
                'learning_rate': self.learning_rate,
                'discount_factor': self.discount_factor,
                'epsilon': self.epsilon,
                'behavior_threshold': self.behavior_threshold
            }
        }
        with open(filename, 'wb') as f:
            pickle.dump(model_data, f)
        print("Model saved successfully!")

    @classmethod
    def load_model(cls, filename='dqn_auth_model.h5'):
        """Load the model"""
        print(f"\nLoading model from {filename}...")
        with open(filename, 'rb') as f:
            model_data = pickle.load(f)

        model = cls(
            learning_rate=model_data['params']['learning_rate'],
            discount_factor=model_data['params']['discount_factor'],
            epsilon=model_data['params']['epsilon'],
            behavior_threshold=model_data['params']['behavior_threshold']
        )

        model.model = model._build_model()
        model.model.set_weights(model_data['model_weights'])
        print("Model loaded successfully!")
        return model

# Example usage:
if __name__ == "__main__":
    print("Loading pre-encoded data...")
    data = pd.read_csv("/content/drive/MyDrive/archive/encoded_data.csv")
    print(f"Data loaded. Total records: {len(data)}")

    # Initialize and train model
    model = DQNAuthorizationModel()
    train_accuracy, test_accuracy = model.train(data, epochs=10)

    # Save model
    model.save_model("dqn_auth_model.h5")

    # Example prediction with encoded values
    loaded_model = DQNAuthorizationModel.load_model("dqn_auth_model.h5")
    prediction = loaded_model.predict(role_encoded=1, file_path_encoded=2, behavior_prob=0.85)
    print("\nExample Prediction:", "Access Granted" if prediction else "Access Denied")

Loading pre-encoded data...
Data loaded. Total records: 1000

=== Training Configuration ===
Learning Rate: 0.001
Initial Epsilon: 1.0
Behavior Threshold: 0.7
Total epochs: 10

Data split: 800 training samples, 200 test samples

Initializing neural network...
Model architecture:
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 64)                256       
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_21 (Dense)            (None, 32)                2080      
                                                                 
 dropout_11 (Dropout)        (None, 32)                0         
                                                                 
 dense_22 (Dense)            (None, 16